In [32]:
import time
import pygame
import math
import random

class Section:
    def __init__(self,p):
        self.p1 = p[0]
        self.p2 = p[1]
        self.p3 = p[2]
        self.p4 = p[3]
    def getArea(self):
        return (1)*(abs(self.p1[0]-self.p4[0]) + abs(self.p2[0]-self.p3[0]))/2
    def getSlopeAndIntercept(self,p1,p2):
        x1 = p1[0]
        y1 = p1[1]
        x2 = p2[0]
        y2 = p2[1]
        slope = (y1-y2)/(x1-x2)
        intercept = y1 - (slope * x1)
        return [slope,intercept]
    def getFuzzyValue(self,x):
        if x < self.p1[0] or x > self.p4[0]:
            return 0
        elif x >= self.p1[0] and x <= self.p2[0] : 
            slopeAndIntercept = self.getSlopeAndIntercept(self.p1,self.p2)
            return slopeAndIntercept[0]*x + slopeAndIntercept[1]; 
        elif x >= self.p2[0] and x <= self.p3[0] : 
            return 1
        elif x >= self.p3[0] and x <= self.p4[0] : 
            slopeAndIntercept = self.getSlopeAndIntercept(self.p3,self.p4)
            return slopeAndIntercept[0]*x + slopeAndIntercept[1]; 


In [33]:
NEGATIVE_MED_P = [[-1000,0],[-800,1],[-200,1],[-5,0]]
NEGATIVE_SMALL_P = [[-100,0],[-80,1],[-20,1],[0,0]]
ZERO_P = [[-10,0],[0,1],[0,1],[10,0]]
POSITIVE_SMALL_P = [[0,0],[20,1],[80,1],[100,0]]
POSITIVE_MED_P = []

angle = 1
angular_vel = 1
angular_acc = 0
angular_vel_scale = 1

negative_med = Section(NEGATIVE_MED_P)
negative_small = Section(NEGATIVE_SMALL_P)
zero = Section(ZERO_P)
positive_small = Section(POSITIVE_SMALL_P)

curr = 0
i=0

In [34]:
pygame.init()

SIZE = 640, 480
screen = pygame.display.set_mode(SIZE)
FPSCLOCK = pygame.time.Clock()
    
startpoint = pygame.math.Vector2(320, 240)
endpoint = pygame.math.Vector2(0, -170)


done = False
while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
    
    effective_angular_vel = angular_vel * angular_vel_scale
    u_zero_angle = zero.getFuzzyValue(angle)
    u_zero_angular_vel = zero.getFuzzyValue(effective_angular_vel)
    u_positive_small_angle = positive_small.getFuzzyValue(angle)
    u_positive_small_angular_vel = positive_small.getFuzzyValue(effective_angular_vel)
    #u_positive_med_angle = positive_med.getFuzzyValue(angle)
    #u_positive_med_anglular_vel = positive_med.getFuzzyValue(effective_angular_vel)
    #u_negative_small_angle = negative_small.getFuzzyValue(angle)
    #u_negative_small_angular_vel = negative_small.getFuzzyValue(effective_angular_vel)
    #u_negative_med_angle = negative_med.getFuzzyValue(angle)
    #u_negative_med_anglular_vel = negative_med.getFuzzyValue(effective_angular_vel)
    
    if angle < 0:
        u_zero_angle = zero.getFuzzyValue(-angle)
        u_positive_small_angle = positive_small.getFuzzyValue(-angle)
        #u_positive_med_angle = positive_med.getFuzzyValue(angle)
        #u_positive_med_anglular_vel = positive_med.getFuzzyValue(effective_angular_vel)
     #   u_negative_small_angle = negative_small.getFuzzyValue(-angle)
    if angular_vel < 0:
        u_zero_angular_vel = zero.getFuzzyValue(-effective_angular_vel)
        u_positive_small_angular_vel = positive_small.getFuzzyValue(-effective_angular_vel)
        
    print("current - ",curr)  
    print("angle - ",angle)
    print("angular_vel - ",angular_vel)
    print("angular_acc - ",angular_acc)
    if angle == 0 and angular_vel == 0 : 
        curr = 0 
    elif angle == 0 and angular_vel > 0 : #-ve small
        print("")
    elif angle == 0 and angular_vel < 0 : #+ve small
        print("")
        
    
    elif angle > 0 and angular_vel == 0 : #-ve small
        print("")
    elif angle > 0 and angular_vel > 0 : #-ve med
        zero_trap_height = min(u_zero_angle,u_zero_angular_vel)
        neg_small_trap_height1 = min(u_zero_angle,u_positive_small_angular_vel)
        neg_small_trap_height2 = min(u_positive_small_angle,u_zero_angular_vel)
        neg_med_trap_height = min(u_positive_small_angle,u_positive_small_angular_vel)
        
        zero_trap_area = zero_trap_height * zero.getArea()
        neg_small_trap_height1_area = neg_small_trap_height1 * negative_small.getArea()
        neg_small_trap_height2_area = neg_small_trap_height2 * negative_small.getArea()
        neg_med_trap_area = neg_med_trap_height * negative_med.getArea()
        
        area_sum = zero_trap_area + neg_small_trap_height1_area + neg_small_trap_height2_area  + neg_med_trap_area
        
        centroid_area_zero = ((zero.p1[0] + zero.p4[0])/2)*zero_trap_area
        centroid_area_neg_small1 = ((negative_small.p1[0] + negative_small.p4[0])/2)*neg_small_trap_height1_area
        centroid_area_negsmall2 = ((negative_small.p1[0] + negative_small.p4[0])/2)*neg_small_trap_height2_area
        centroid_area_zero = ((negative_med.p1[0] + negative_med.p4[0])/2)* neg_med_trap_area 
        
        curr = (centroid_area_zero + centroid_area_neg_small1 + centroid_area_negsmall2 + centroid_area_zero)/area_sum
        
        
        
        
    elif angle > 0 and angular_vel < 0 : 
        curr = 0
    
    
    elif angle < 0 and angular_vel == 0 : #+ve small
        print("")
    elif angle < 0 and angular_vel > 0 : 
        curr = 0
    elif angle < 0 and angular_vel < 0 : #+ve med
        
        zero_trap_height = min(u_zero_angle,u_zero_angular_vel)
        neg_small_trap_height1 = min(u_zero_angle,u_positive_small_angular_vel)
        neg_small_trap_height2 = min(u_positive_small_angle,u_zero_angular_vel)
        neg_med_trap_height = min(u_positive_small_angle,u_positive_small_angular_vel)
        
        
        zero_trap_area = zero_trap_height * zero.getArea()
        neg_small_trap_height1_area = neg_small_trap_height1 * negative_small.getArea()
        neg_small_trap_height2_area = neg_small_trap_height2 * negative_small.getArea()
        neg_med_trap_area = neg_med_trap_height * negative_med.getArea()
        
        area_sum = zero_trap_area + neg_small_trap_height1_area + neg_small_trap_height2_area  + neg_med_trap_area
        
        centroid_area_zero = ((zero.p1[0] + zero.p4[0])/2)*zero_trap_area
        centroid_area_neg_small1 = ((negative_small.p1[0] + negative_small.p4[0])/2)*neg_small_trap_height1_area
        centroid_area_negsmall2 = ((negative_small.p1[0] + negative_small.p4[0])/2)*neg_small_trap_height2_area
        centroid_area_zero = ((negative_med.p1[0] + negative_med.p4[0])/2)* neg_med_trap_area 
        
        curr = (centroid_area_zero + centroid_area_neg_small1 + centroid_area_negsmall2 + centroid_area_zero)/area_sum
        curr = -curr
        
    if curr > 20 :
        curr = 20
    if curr < -20 :
        curr = -20
    if angle > 10 : 
        angle = 10
        angular_vel = -angular_vel
        
    if angle < -10:
        angle = -10
        angular_vel = -angular_vel
    angular_acc = angular_acc + curr/150
    angular_vel = angular_vel + angular_acc/100
    
    prev_angle = angle
    if angular_vel > 0 :
        angle = angle + angular_vel*angular_vel/300
    else :
        angle = angle - angular_vel*angular_vel/300
        
    if prev_angle*angle < 0:
        angular_acc = 0
   
            
    current_endpoint = startpoint + endpoint.rotate(angle)

    screen.fill((0, 0, 0))
    pygame.draw.line(
        screen, pygame.Color("red"), startpoint, current_endpoint, 2)

    pygame.display.flip()  
    FPSCLOCK.tick(60)

pygame.quit()

current -  0
angle -  1
angular_vel -  1
angular_acc -  0
current -  -20
angle -  1.0033244503703704
angular_vel -  0.9986666666666667
angular_acc -  -0.13333333333333333
current -  -20
angle -  1.0066311703703705
angular_vel -  0.996
angular_acc -  -0.26666666666666666
current -  -20
angle -  1.009911383703704
angular_vel -  0.992
angular_acc -  -0.4
current -  -20
angle -  1.0131564207407409
angular_vel -  0.9866666666666667
angular_acc -  -0.5333333333333333
current -  -20
angle -  1.0163577540740742
angular_vel -  0.98
angular_acc -  -0.6666666666666666
current -  -20
angle -  1.0195070340740742
angular_vel -  0.972
angular_acc -  -0.7999999999999999
current -  -20
angle -  1.0225961244444446
angular_vel -  0.9626666666666667
angular_acc -  -0.9333333333333332
current -  -20
angle -  1.0256171377777779
angular_vel -  0.952
angular_acc -  -1.0666666666666667
current -  -20
angle -  1.0285624711111112
angular_vel -  0.94
angular_acc -  -1.2
current -  -20
angle -  1.0314248414814815


current -  0
angle -  0.804923934814814
angular_vel -  -2.2760000000000016
angular_acc -  -5.200000000000003
current -  0
angle -  0.7868586548148139
angular_vel -  -2.3280000000000016
angular_acc -  -5.200000000000003
current -  0
angle -  0.7679773214814806
angular_vel -  -2.3800000000000017
angular_acc -  -5.200000000000003
current -  0
angle -  0.7482619081481472
angular_vel -  -2.4320000000000017
angular_acc -  -5.200000000000003
current -  0
angle -  0.7276943881481471
angular_vel -  -2.4840000000000018
angular_acc -  -5.200000000000003
current -  0
angle -  0.7062567348148138
angular_vel -  -2.536000000000002
angular_acc -  -5.200000000000003
current -  0
angle -  0.6839309214814804
angular_vel -  -2.588000000000002
angular_acc -  -5.200000000000003
current -  0
angle -  0.6606989214814804
angular_vel -  -2.640000000000002
angular_acc -  -5.200000000000003
current -  0
angle -  0.636542708148147
angular_vel -  -2.692000000000002
angular_acc -  -5.200000000000003
current -  0
ang

current -  20
angle -  -1.7187696533333388
angular_vel -  -1.552000000000001
angular_acc -  7.4666666666666766
current -  20
angle -  -1.7260315733333387
angular_vel -  -1.4760000000000009
angular_acc -  7.60000000000001
current -  20
angle -  -1.7325524681481534
angular_vel -  -1.3986666666666674
angular_acc -  7.733333333333344
current -  20
angle -  -1.7383604681481535
angular_vel -  -1.3200000000000007
angular_acc -  7.866666666666678
current -  20
angle -  -1.743485801481487
angular_vel -  -1.2400000000000007
angular_acc -  8.00000000000001
current -  20
angle -  -1.7479608296296352
angular_vel -  -1.1586666666666672
angular_acc -  8.133333333333344
current -  20
angle -  -1.7518200829629684
angular_vel -  -1.0760000000000005
angular_acc -  8.266666666666676
current -  20
angle -  -1.7551002962963018
angular_vel -  -0.9920000000000004
angular_acc -  8.40000000000001
current -  20
angle -  -1.75784044444445
angular_vel -  -0.906666666666667
angular_acc -  8.533333333333342
current 

angular_acc -  -1.3333333333333333
current -  -20
angle -  1.0398690370370356
angular_vel -  5.260000000000003
angular_acc -  -1.4666666666666666
current -  -20
angle -  1.1315341570370356
angular_vel -  5.244000000000003
angular_acc -  -1.5999999999999999
current -  -20
angle -  1.222594305185184
angular_vel -  5.22666666666667
angular_acc -  -1.7333333333333332
current -  -20
angle -  1.313005185185184
angular_vel -  5.208000000000004
angular_acc -  -1.8666666666666665
current -  -20
angle -  1.4027229985185174
angular_vel -  5.188000000000004
angular_acc -  -1.9999999999999998
current -  -20
angle -  1.491704479999999
angular_vel -  5.1666666666666705
angular_acc -  -2.1333333333333333
current -  -20
angle -  1.5799069333333324
angular_vel -  5.144000000000004
angular_acc -  -2.2666666666666666
current -  -20
angle -  1.667288266666666
angular_vel -  5.120000000000004
angular_acc -  -2.4
current -  -20
angle -  1.7538070281481475
angular_vel -  5.0946666666666705
angular_acc -  -2.5

current -  0
angle -  4.462415235555561
angular_vel -  -0.471999999999997
angular_acc -  -11.999999999999996
current -  0
angle -  4.461247022222228
angular_vel -  -0.591999999999997
angular_acc -  -11.999999999999996
current -  0
angle -  4.459557208888895
angular_vel -  -0.711999999999997
angular_acc -  -11.999999999999996
current -  0
angle -  4.457249795555562
angular_vel -  -0.831999999999997
angular_acc -  -11.999999999999996
current -  0
angle -  4.454228782222229
angular_vel -  -0.951999999999997
angular_acc -  -11.999999999999996
current -  0
angle -  4.450398168888896
angular_vel -  -1.071999999999997
angular_acc -  -11.999999999999996
current -  0
angle -  4.445661955555562
angular_vel -  -1.1919999999999968
angular_acc -  -11.999999999999996
current -  0
angle -  4.439924142222229
angular_vel -  -1.3119999999999967
angular_acc -  -11.999999999999996
current -  0
angle -  4.433088728888896
angular_vel -  -1.4319999999999966
angular_acc -  -11.999999999999996
current -  0
ang

...

angular_acc -  0
current -  20
angle -  -0.9976566992592524
angular_vel -  -14.95066666666666
angular_acc -  0.13333333333333333
current -  20
angle -  -1.742465712592585
angular_vel -  -14.947999999999993
angular_acc -  0.26666666666666666
current -  20
angle -  -2.486876165925918
angular_vel -  -14.943999999999994
angular_acc -  0.4
current -  20
angle -  -3.2307553718518434
angular_vel -  -14.938666666666661
angular_acc -  0.5333333333333333
current -  20
angle -  -3.973970785185176
angular_vel -  -14.931999999999995
angular_acc -  0.6666666666666666
current -  20
angle -  -4.716390038518509
angular_vel -  -14.923999999999996
angular_acc -  0.7999999999999999
current -  20
angle -  -5.4578809777777675
angular_vel -  -14.914666666666662
angular_acc -  0.9333333333333332
current -  20
angle -  -6.198311697777767
angular_vel -  -14.903999999999995
angular_acc -  1.0666666666666667
current -  20
angle -  -6.937550577777766
angular_vel -  -14.891999999999994
angular_acc -  1.2
current - 

angle -  1.6374216592592354
angular_vel -  15.349333333333316
angular_acc -  -0.26666666666666666
current -  -20
angle -  2.4223525096296044
angular_vel -  15.345333333333317
angular_acc -  -0.4
current -  -20
angle -  3.2067378429629363
angular_vel -  15.339999999999984
angular_acc -  -0.5333333333333333
current -  -20
angle -  3.9904415466666383
angular_vel -  15.333333333333318
angular_acc -  -0.6666666666666666
current -  -20
angle -  4.773327685925896
angular_vel -  15.325333333333319
angular_acc -  -0.7999999999999999
current -  -20
angle -  5.555260539259227
angular_vel -  15.315999999999985
angular_acc -  -0.9333333333333332
current -  -20
angle -  6.33610463407404
angular_vel -  15.305333333333317
angular_acc -  -1.0666666666666667
current -  -20
angle -  7.115724782222187
angular_vel -  15.293333333333317
angular_acc -  -1.2
current -  -20
angle -  7.893986115555518
angular_vel -  15.279999999999983
angular_acc -  -1.3333333333333333
current -  -20
angle -  8.670754121481442


current -  20
angle -  -9.581866690370314
angular_vel -  -15.62933333333331
angular_acc -  1.4666666666666666
current -  20
angle -  -10.39445394962957
angular_vel -  -15.61333333333331
angular_acc -  1.5999999999999999
current -  20
angle -  -9.185607531851854
angular_vel -  15.630666666666643
angular_acc -  1.7333333333333332
current -  0
angle -  -8.369407851851857
angular_vel -  15.647999999999977
angular_acc -  1.7333333333333332
current -  0
angle -  -7.551398957037045
angular_vel -  15.66533333333331
angular_acc -  1.7333333333333332
current -  0
angle -  -6.731578844444455
angular_vel -  15.682666666666643
angular_acc -  1.7333333333333332
current -  0
angle -  -5.909945511111124
angular_vel -  15.699999999999976
angular_acc -  1.7333333333333332
current -  0
angle -  -5.086496954074089
angular_vel -  15.71733333333331
angular_acc -  1.7333333333333332
current -  0
angle -  -4.2612311703703885
angular_vel -  15.734666666666643
angular_acc -  1.7333333333333332
current -  0
angl

current -  0
angle -  1.4260025777778094
angular_vel -  -16.11599999999997
angular_acc -  -1.7333333333333332
current -  0
angle -  0.5583877629629976
angular_vel -  -16.133333333333304
angular_acc -  -1.7333333333333332
current -  0
angle -  -0.31109234962959176
angular_vel -  -16.150666666666638
angular_acc -  0
current -  20
angle -  -1.1804289066666256
angular_vel -  -16.149333333333303
angular_acc -  0.13333333333333333
current -  20
angle -  -2.049478388148104
angular_vel -  -16.146666666666636
angular_acc -  0.26666666666666666
current -  20
angle -  -2.9180973451851377
angular_vel -  -16.142666666666635
angular_acc -  0.4
current -  20
angle -  -3.786142435555505
angular_vel -  -16.137333333333302
angular_acc -  0.5333333333333333
current -  20
angle -  -4.653470459259205
angular_vel -  -16.130666666666635
angular_acc -  0.6666666666666666
current -  20
angle -  -5.519938394074017
angular_vel -  -16.122666666666635
angular_acc -  0.7999999999999999
current -  20
angle -  -6.385

angular_vel -  16.37466666666659
angular_acc -  -1.4666666666666666
current -  -20
angle -  9.10448682074075
angular_vel -  -16.39066666666659
angular_acc -  -1.5999999999999999
current -  0
angle -  8.207224450370388
angular_vel -  -16.406666666666588
angular_acc -  -1.5999999999999999
current -  0
angle -  7.308211182222249
angular_vel -  -16.422666666666586
angular_acc -  -1.5999999999999999
current -  0
angle -  6.407445309629665
angular_vel -  -16.438666666666585
angular_acc -  -1.5999999999999999
current -  0
angle -  5.50492512592597
angular_vel -  -16.454666666666583
angular_acc -  -1.5999999999999999
current -  0
angle -  4.600648924444498
angular_vel -  -16.47066666666658
angular_acc -  -1.5999999999999999
current -  0
angle -  3.694614998518582
angular_vel -  -16.48666666666658
angular_acc -  -1.5999999999999999
current -  0
angle -  2.7868216414815548
angular_vel -  -16.502666666666578
angular_acc -  -1.5999999999999999
current -  0
angle -  1.87726714666675
angular_vel -  

current -  -8.924019607843139
angle -  1.0153346677059305
angular_vel -  0.5537990196078431
angular_acc -  -0.44620098039215694
current -  -7.663407476977831
angle -  1.016790374443607
angular_vel -  0.17062864575895154
angular_acc -  -0.3831703738488915
current -  -5.935809316564023
angle -  1.0159945342014478
angular_vel -  -0.1261618200692496
angular_acc -  -0.2967904658282011
current -  0
angle -  1.0151986939592885
angular_vel -  -0.1261618200692496
angular_acc -  0.0
current -  0
angle -  1.0144028537171292
angular_vel -  -0.1261618200692496
angular_acc -  0.0
current -  0
angle -  1.01360701347497
angular_vel -  -0.1261618200692496
angular_acc -  0.0
current -  0
angle -  1.0128111732328107
angular_vel -  -0.1261618200692496
angular_acc -  0.0
current -  0
angle -  1.0120153329906514
angular_vel -  -0.1261618200692496
angular_acc -  0.0
current -  0
angle -  1.011219492748492
angular_vel -  -0.1261618200692496
angular_acc -  0.0
current -  0
angle -  1.0104236525063328
angular_v

KeyboardInterrupt: 